In [ ]:
import netgen.gui
from ngsolve import *
import numpy as np
import scipy.sparse as sp
from math import pi
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [ ]:
import netgen.geom2d as geom2dim
geo = geom2dim.SplineGeometry()
a = 1
points = [(0,0),(a,0),(a,a),(0,a)]
p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in points]
curves = [['line',p1,p2],['line',p2,p3],['line',p3,p4],['line',p4,p1]]
[geo.Append(c) for c in curves]

mesh = Mesh(geo.GenerateMesh(maxh = 0.1))


piha = a/2
shift = 0
source = exp(-70**2*((x-piha-shift)*(x-piha-shift)+(y-piha-shift)*(y-piha-shift)))
Draw(source,mesh,name = "pulse")

In [ ]:
x_0 = 0.3
y_0 = 0.6
factor = 25

func = exp(-factor*((x-x_0)**2 + (y-y_0)**2))

f = LinearForm(fes)
f += func * v * dx
f.Assemble()

omega = Parameter(0)
a = BilinearForm(fes)
a += (grad(u)*grad(v) - omega*omega *u*v) * dx
a.Assemble()


gfu = GridFunction(fes)
if visual: Draw(gfu)

In [ ]:
if visual: Draw(gfu)
stepsize= 0.5
beginning = 0
end = 20
red_param = np.arange(beginning,end, stepsize)
visual = False

In [ ]:
# choosing_stepsize = 0.01
# beginning = 0
# end = 20
# choosing_param = np.arange(beginning,end, choosing_stepsize)
# red_param = []
# res = lambda 
# for _omega in choosing_param:

In [ ]:
def GramSchmidt (V, u):
    if V ==[]:
        z = u
    else:
        #computational expensive
        V_mat = np.array(V)
        z = u - np.transpose(V_mat).dot(V_mat).dot(u)
    z = z/np.linalg.norm(z)
    V += [z]
    return V

In [ ]:
dim_orig= len(gfu.vec)# dim of original space
dim_red = len(red_param)  # dim of reduced space
norm_of_solutions_orig = [] # l2 norm 
V_tmp = np.zeros((dim_orig,dim_red)) # reduced basis
# V = []
sol = []
## start with this rb
for n in range(0, dim_red):
    _omega = red_param[n]
    ## compute FEM solution for parameter _omega
    omega.Set(_omega)
    a.Assemble()
    gfu.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") * f.vec
    nof = Integrate(gfu*gfu, mesh)
    norm_of_solutions_orig += [nof]
    V_tmp[:,n] = np.array(gfu.vec.data) 
    sol += [gfu.vec]
#     GramSchmidt(V, np.array(gfu.vec.data))

    if visual:
        Redraw()
        input("norm of solution {}: {}".format(_omega, nof))

        
# V = np.transpose(np.array(V))
# print(V.shape)

q, r = np.linalg.qr(V_tmp)
V = V_tmp.dot(np.linalg.inv(r))

In [ ]:
# K_red = np.zeros((dim_red, dim_red))
# M_red = np.zeros((dim_red, dim_red))
# P_red = np.zeros(dim_red)


# tmpi = GridFunction(fes)
# tmpj = GridFunction(fes)
# for i in range(0, dim_red):
#     for j in range(0, dim_red):
#         tmpi.vec.FV().NumPy()[:] = V[:,i]
#         tmpj.vec.FV().NumPy()[:] = V[:,j]
#         M_red[i,j] = Integrate(tmpi*tmpj, mesh)
#         K_red[i,j] = Integrate(grad(tmpi)*grad(tmpj), mesh)
#     P_red[i] = Integrate(func*tmpi, mesh)


In [ ]:
import scipy.sparse as sparse

k_blf = BilinearForm(fes)
k_blf += grad(u)*grad(v)*dx
k_blf.Assemble()
rows,cols,vals = k_blf.mat.COO()
K_orig = sparse.csr_matrix((vals,(rows,cols)))

m_blf = BilinearForm(fes)
m_blf += u*v * dx
m_blf.Assemble()
rows,cols,vals = m_blf.mat.COO()
M_orig = sparse.csr_matrix((vals,(rows,cols)))

## why does this not work? Gives:
## ValueError: operands could not be broadcast together with shapes (100,2976) (2976,100)
# K_red = V.T*m_blf.mat*V

K_red = np.transpose(V).dot(K_orig.dot(V))
M_red = np.transpose(V).dot(M_orig.dot(V))
P_red = np.transpose(V).dot(f.vec.data)



In [ ]:
reduced_stepsize = stepsize
comp_param = np.arange(beginning,end, reduced_stepsize)
norm_of_solutions_red = [] 
residual = []

ured = GridFunction(fes)
visual = False

if visual: Draw(ured)
j = 0
for _omega in comp_param:
    ## compute reduced solution for parameter _omega
#     omega.Set(_omega)
#     a.Assemble()
#     ## just for testing
#     rows,cols,vals = a.mat.COO()
#     A = sparse.csr_matrix((vals,(rows,cols)))
    
    A = K_orig-_omega*_omega*M_orig
    Ainv = np.linalg.inv(A)
    red_sol_vec = np.matmul(Ainv, P_red)
    ured.vec.FV().NumPy()[:] = V.dot(red_sol_vec)[:]
    nof = Integrate(ured*ured, mesh)
    norm_of_solutions_red += [nof]
    ## compute resiudal (euklidean)
    print(np.linalg.norm(np.array(sol[j])+np.array(ured.vec.data)))
    
#     print(np.linalg.norm(np.array(ured.vec)-np.array(sol[j])))
#     print(sol[j])
    res = np.linalg.norm((K_orig-_omega*_omega*M_orig).dot(np.array(ured.vec.data))-f.vec.data)
    #res = np.linalg.norm(A.dot(np.array(ured.vec))-np.array(f.vec.data))
    ## just for testing
#     res = np.linalg.norm((K_orig-_omega*_omega*M_orig).dot(np.array(V[:,j]))-f.vec.data)
    residual += [res]
    #print("norm of solution {}: {}".format(_omega, nof))
    j += 1
    if visual:
        Redraw()
        input("norm of solution {}: {}".format(_omega, nof))


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

ax.plot(red_param, norm_of_solutions_orig)

plt.yscale('log')
ax.set_title("original space with stepsize {}".format(stepsize))

fig, ax = plt.subplots()
ax.semilogy(comp_param, norm_of_solutions_red)
plt.yscale('log')
ax.set_title("reduced space with stepsize {}".format(reduced_stepsize))

if stepsize == reduced_stepsize:
    fig, ax = plt.subplots()
    ax.semilogy(comp_param, np.array(norm_of_solutions_red)-np.array(norm_of_solutions_orig))
    plt.yscale('log')
    ax.set_title("norm difference")


fig, ax = plt.subplots()
ax.semilogy(comp_param, residual)
plt.yscale('log')
ax.set_title("residual with stepsize {}".format(reduced_stepsize))
plt.show()

